# Tổng quan báo cáo

Sau khi train qua khoảng 35 model (bao gồm cách thay đổi kiến trúc model hoặc cách biến đổi tiền xử lý dữ liệu) em đã chọn ra 5 model có thể đưa ra kết luận quan trọng

## Về dữ liệu: 

Từ bộ dữ liệu gốc của thầy em phân chia train/valid/test theo tỷ lệ 80:10:10.

## Về model tốt nhất hiện tại 

(em tạm ký hiệu H97), trên tập test (10% số lượng ảnh của bộ dữ liệu ban đầu) đang thu được kết quả là: 

- 87,6% độ chính xác accuracy; 
- bên cạnh đó f1-score (đánh giá sự chính xác của 3 nhãn phân loại do bài toán có nhãn B2 mất cân bằng lớn so với các nhãn B5 B6) đạt được 87,41% trung bình cho 3 nhãn;
- đánh giá bằng đường ROC AUC cũng đạt kết quả khả quan khi 3 nhãn có diện tích dưới dường cong lần lượt là: B2 - 0.93, B5 - 0.94, B6 - 0.95;

## Về các kết luận từ 7 biểu đồ em chắt lọc:

- biểu đồ đầu tiên là của model H97 trên tập valid (10% lượng ảnh gốc ban đầu) (đạt 90% accuracy)
- biểu đồ tiếp theo là của H97 trên tập train (80% lượng ảnh gốc ban đầu) (đạt 97% accuracy)
- biểu đồ thứ 3 là của H97 trên tập test (10% lượng ảnh gốc ban đầu) (đạt 87,6% accuracy)

$\to$ 3 biểu đồ này cho thấy kết quả của model tốt nhất em thu được hiện tại (model H97)
- biểu đồ thứ 4 là của model ViT, hiện ViT đang bày tỏ sự ấn tượng  trong các bài toán phân loại ảnh. Tuy nhiên sau nhiều lần thử nghiệm em chưa thấy ViT đạt kết quả tốt do dữ liệu huấn luyện để ViT có được kết quả tốt yêu cầu rất lớn. Mặt khác do phần cứng hạn chế nên em chỉ mới train ViT được 10 epoch
- biểu đồ thứ 5 là của model phân loại nhị phân 2 nhãn B5 B6, có thể thấy việc cố phân loại 2 nhãn B5 B6 riêng ra không tốt hơn việc phân loại 3 nhãn B2, B5, B6
- biểu đồ thứ 6 là của model phân loại nhị phân 2 nhãn B2 (lành tính), B5B6 (ác tính). Phân loại nhị phân như này đạt kết quả rất ấn tượng.  Tuy nhiên các bài báo cũ trước đó đã làm qua việc phân loại nhị phân lành tính, ác tính này rồi
- biểu đồ thứ 7 là phân loại 3 nhãn B2 B5 B6 - em đã re train lại toàn bộ mạng do dữ liệu tế bào học là quá khác biệt so với các ảnh trong tập image net dùng cho các model pretrained thường thấy. Do vậy em từ bỏ hướng dùng pretrained và quyết định train lại toàn bộ mạng


# Về cách em triển khai model H97:

- model H97 bao gồm 3 module (module 1 là model xác định vùng quan tâm của ảnh), module 2 là model CNN nhận đầu vào là các ảnh cắt tổng hợp từ module 1, module 3 là 1 model ANN nhận input là vector 54 chiều (thu được và duỗi ra từ module 2) và đầu ra là 3 (xác suất thuộc về 3 nhãn B2, B5, B6)
- sau khi chuyển toàn bộ source code từ Tensorflow sang PyTorch em đã quyết định xây dựng lại module 1 theo hướng đơn giản hơn.  Cụ thể thay vì xác định từng tế bào riêng lẻ rồi dùng giải thuật phâncụm, cắt theo mật độ; em đã nhận thấy sự đồng nhất trong việc gán nhãn các cụm tế bào và đã gán nhãn lại dữ liệu cũng như xây dựng lại để xác định vùng cụm tế bào. Từ đó giảm khối lượng tính toán,mà hiệu quả không thay đổi
- về module 2 của model H97, bản chất nó là mạng EfficientNetB0 (em thấy mạng này nhẹ và dễ train lại toàn bộ mạng hơn so với các kiến trúc khác). Mạng trích suất đặc trưng sẽ qua các lớp dense (9,73,3). Bằng cách cho 12 ảnh (cắt theo lưới từ ảnh gốc), 5 ảnh cắt thu được từ các cụm quan trọng của ảnh, 1 ảnh gốc (resize về 224x224). Như vậy đầu ra của module 2 là vector 18*3=54 chiều.
- về module 3 của model H97, đây là 1 ANN đơn giản để tạo quan hệ phi tuyến của input với output, trong đó trọng số từ các đặc trưng của 5 ảnh cắt vùng quan trọng được đặt cao hơn 13 ảnh còn lại (gấp khoảng 5 lần), 12 ảnh cắt theo lưới có trọng số cao hơn ảnh gốc resize 224x224
